# Ex3 : Compressible Euler simulation - CPU Multi partition version

### Set work directory

In [ ]:
DGLM_DIR = "/home/user/0.4.1-OpenSource"
WORK_DIR = "ex3.run.cpu.uniform.npart-3"

### Set modules

In [ ]:
%load_ext autoreload
%autoreload all

### Initialize DGLM object <a name='init_obj'></a>

In [ ]:
N    = 5
Nvar = 5  
tmax = 2
mpisize = npart = 3


x1, x2, nx = 0, 1, 7  # Nelem=1296, h=0.06904 (cf, nx=6: Nelem=750, h=0.8284)
y1, y2, ny = 0, 1, 7
z1, z2, nz = 0, 1, 7
bc = {'x':'pbc', 'y':'pbc', 'z':'pbc'}
mesh_info = {"type": "uniform", "param": [nx, ny, nz, x1, x2, y1, y2, z1, z2, bc]}


dglm_param = {
        'CFL'      : 0.8,
        'gamma'    : 1.4,
        'tau_scale': 1.2,
        'interpN'  : 2,
        'dtype'    : 'f8'}

print(f"DGLM_DIR={DGLM_DIR}")
print(f"WORK_DIR={WORK_DIR}")
        
import sys
sys.path.append(f"{DGLM_DIR}/dim3/src/python")
from dglm_3d_launcher import DGLM3DMpiCPU

dg_list = []
for myrank in range(mpisize):
    dg = DGLM3DMpiCPU(myrank, mpisize, DGLM_DIR, WORK_DIR, N, Nvar, tmax, dglm_param)
    dg_list.append(dg)    
    dg.init_mesh(mesh_info, verbose=1)
    dg.make_work_dir()
    dg.save_to_binaries(verbose=1)

print()
dg = dg_list[0]
prefix = 'euler_mpi_'
suffix = '.cpp'
names = ['data', 'comm', 'dglm', 'time', 'init', 'main']
filenames = [prefix+name+suffix for name in names]
dg.save_to_header([fname for fname in filenames if 'main' not in fname], verbose=1)
dg.link_codes(filenames, verbose=1)    
dg.build([fname for fname in filenames if 'main' in fname], use_sanitize=False, verbose=1)

dg.check_mem_size(prefix+'data'+suffix, verbose=1)
hmin, hmax = dg.mesh.calc_spatial_h(verbose=1)

### Memory size

DGLM|Mesh|Nelem|Npart|h|N|NP|NFACE|NFP|Nvar|total bytes
:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:
0.3.3|Gmsh|768|-|0.03655|5|56|4|21|5|17.14 MB
0.3.3|Uniform|750|-|0.08284|5|56|4|21|5|16.74 MB
0.4.0|Uniform|1296|1|0.06904|5|56|4|21|5|28.84 MB
0.4.0|Uniform|1296(648)|2|0.06904|5|56|4|21|5|30.30 MB (15.15 MB)
0.4.0|Uniform|1296(432)|3|0.06904|5|56|4|21|5|30.78 MB (10.26 MB)
0.4.0|Gmsh|768(256)|3|0.03742|5|56|4|21|5|18.75 MB (6.25 MB)

### Run

In [ ]:
num_cpu_cores = 64
num_omp_threads = num_cpu_cores//dg.npart
tstep, avg_dt, elapsed_time = dg.run('euler_mpi_main', omp_threads=num_omp_threads, verbose=1)

### L2 error (pointwise)

In [ ]:
import numpy as np

def ref_solution(dg, t):
    x = dg.mesh.high_PX.ravel()
    y = dg.mesh.high_PY.ravel()
    z = dg.mesh.high_PZ.ravel()
        
    vx = np.ones_like(x)
    vy = -0.5*np.ones_like(y)
    vz = np.ones_like(z)

    xvt = ((x-vx*t) + 1)%1 - 1
    yvt = ((y-vy*t) + 1)%1 - 1
    zvt = ((z-vz*t) + 1)%1 - 1
    u1_ref = 1 + 0.2*np.sin(2*np.pi*(xvt+yvt+zvt))

    return u1_ref

In [ ]:
from glob import glob

l2_list = []
for dg in dg_list:
    out_fpaths = sorted(glob(dg.dataout_dir + "/*.bin"))
    u_numeric = np.fromfile(out_fpaths[-1])
    u_ref = ref_solution(dg, t=dg.tmax)
    l2 = dg.l2_error(u_ref, u_numeric, apply_sqrt=False)
    l2_list.append(l2)
    
print(np.sqrt(np.sum(l2_list)))

DGLM|exp|Mesh|Nelem|Npart|N|CFL|h|dt|tmax|tstep|L2 error
:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:
0.3.3|3.16|Gmsh|768|-|5|0.8|0.03655|0.00066|2.0|2961|3.238790136116263e-05
0.3.3|3.16|Uniform|750|-|5|0.8|0.08284|0.00181|2.0|1098|7.674679246306467e-05
0.4.0|3.16|Uniform|1296|1|5|0.8|0.06904|0.00149|2.0|1318|2.429869462976010e-05
0.4.0|3.18|Uniform|1296|2|5|0.8|0.06904|0.00149|2.0|1318|2.429869462976010e-05
0.4.0|3.18|Uniform|1296|3|5|0.8|0.06904|0.00149|2.0|1318|2.429869462976010e-05
0.4.0|3.18|Gmsh|768|3|5|0.8|0.03742|0.00066|2.0|2961|3.238790136116263e-05

### Plot snapshot <a name='plot_snapshot'></a>

In [ ]:
import sys
sys.path.append(DGLM_DIR + "/utils/src/python")
from plot3d import Plot3DTetra

In [ ]:
p3t = Plot3DTetra(dg_list, res='low', backend='trame', use_xvfb=True, file_exist=True)
u, tstep = p3t.read_output_file(seq=-1)
print(f"tstep={tstep}")
pl, pvmesh = p3t.plot(u, 'u')
pl.add_mesh(pvmesh, colormap='CET_L18', opacity='sigmoid')
#contours = pvmesh.contour(np.linspace(0, 1, 6), scalars='u')
#pl.add_mesh(pvmesh.outline(), color="k")
#pl.add_mesh(contours, opacity=0.9, clim=[-1,1])
pl.show()

### Plot animation <a name='plot_ani'></a>

In [ ]:
p3t = Plot3DTetra(dg_list, res='high', backend='trame', use_xvfb=True, file_exist=True)
u0, tstep = p3t.read_output_file(0)
pl, pvmesh = p3t.plot(u0, 'u')
pl.open_movie(WORK_DIR + "/u1.mp4", framerate=12, quality=5)
#pl.open_gif(WORK_DIR + "/u1.gif", fps=12)

pl.add_mesh(pvmesh, scalars="u", colormap='CET_L18', opacity='sigmoid')
#contours = pvmesh.contour(np.linspace(0, 1, 6), scalars='u')
#pl.add_mesh(contours, opacity=0.9, clim=[-1,1])
#pl.add_mesh(pvmesh.outline(), color="k")
pl.show_axes()

# camera angle
num_files = p3t.get_num_output_files()
ang = 360/(num_files-1)

# Update scalars on each frame
for seq in range(1, num_files):
    u, tstep = p3t.read_output_file(seq)
    p3t.set_point_data(u, 'u')
    #contours.copy_from(pvmesh.contour(np.linspace(0, 1, 6), scalars='u'))
    
    pl.add_text(f"tstep={tstep}", name='time-label')
    pl.camera.azimuth = 240  # ang
    pl.write_frame()  # Write this frame

# Be sure to close the plotter when finished
pl.close()

In [ ]:
from IPython.display import Video
Video(WORK_DIR+"/u1.mp4", width=800, html_attributes="controls loop autoplay")